In [ ]:
%pip install gdown
import gdown

url = "https://drive.google.com/drive/folders/12F9epgBqkVCww8VcBeD2S2CIfaLNe0zC?usp=share_link"
gdown.download_folder(url, quiet=True)

In [ ]:
!pip install tqdm
!pip install dlib

In [ ]:
# 23032 images for each feature

import os
import cv2
import math
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import numpy as np
import shutil
import zipfile
import dlib
from tqdm import tqdm
from google.colab.patches import cv2_imshow

shape_predictor_path = "/content/biometric_systems_project/shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector() # HOG-based detector
predictor = dlib.shape_predictor(shape_predictor_path)

def shape_to_normal(shape):
  shape_normal = []
  # shape[0]:shape[16] = jaw line
  # shape[17]:shape[21] = left eyebrow
  # shape[22]:shape[26] = right eyebrow
  # shape[27]:shape[35] = nose (27-30 nose bridge + 31-35 lower nose)
  # shape[36]:shape[41] = left eye
  # shape[42]:shape[47] = right eye
  # shape[48]:shape[59] = outer lip
  # shape[60]:shape[67] = inner lip
  for i in range(0, shape.num_parts):
    shape_normal.append((i, (shape.part(i).x, shape.part(i).y)))
  return shape_normal

def get_eyes_nose_dlib(shape):
  nose = shape[30][1]
  left_eye_x = (shape[36][1][0] + shape[39][1][0]) // 2
  left_eye_y = (shape[36][1][1] + shape[39][1][1]) // 2
  right_eyes_x = (shape[42][1][0] + shape[45][1][0]) // 2
  right_eyes_y = (shape[42][1][1] + shape[45][1][1]) // 2

  return nose, (left_eye_x, left_eye_y), (right_eyes_x, right_eyes_y)

def rotate_points(shape, rotation_matrix):
  points = []
  for i in range(0, 68):
    x,y = shape[i][1]
    points.append((x, y, 1))
  points = np.array(points)
  rotated_points = np.dot(rotation_matrix, points.T).T
  return np.array([(int(p[0]), int(p[1])) for p in rotated_points])

def get_eyes_in_image(image, points):
  height, width = image.shape[:2]
  top = points[17:27][np.argmin(points[17:27, 1])]
  bottom = points[29]
  min_x, min_y = np.min(points[36:48, 0]), np.min(points[36:48, 1])
  max_x, max_y = np.max(points[36:48, 0]), np.max(points[36:48, 1])
  min_x, min_y = max(0, min_x - 10), max(top[1], min_y - 10)
  max_x, max_y = min(width, max_x + 10), min(bottom[1], max_y + 10)
  image = image[min_y:max_y, min_x:max_x]
  return image

def get_nose_in_image(image, points):
  height, width = image.shape[:2]
  top = points[36:48][np.argmin(points[36:48, 1])]
  bottom = points[48:68][np.argmax(points[48:68, 1])]
  min_x, min_y = np.min(points[27:36, 0]), np.min(points[27:36, 1])
  max_x, max_y = np.max(points[27:36, 0]), np.max(points[27:36, 1])
  min_x, min_y = max(0, min_x - 5), max(top[1], min_y - 5)
  max_x, max_y = min(width, max_x + 5), min(bottom[1], max_y + 5)
  image = image[min_y:max_y, min_x:max_x]
  return image

def get_mouth_in_image(image, points):
  height, width = image.shape[:2]
  top = points[27:36][np.argmin(points[27:36, 1])]
  min_x, min_y = np.min(points[48:68, 0]), np.min(points[48:68, 1])
  max_x, max_y = np.max(points[48:68, 0]), np.max(points[48:68, 1])
  min_x, min_y = max(0, min_x - 10), max(top[1], min_y - 10)
  max_x, max_y = min(width, max_x + 10), min(height, max_y + 10)
  image = image[min_y:max_y, min_x:max_x]
  return image

def align_face(image, file_name):
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  faces = detector(gray)
  if len(faces) > 0:
    for face in faces:
    # Extract face bounding box coordinates
      shape = predictor(gray, face)

      # Get facial landmarks for the detected face
      shape = shape_to_normal(shape)

      # Get the nose, left eye, and right eye positions
      nose, left_eye, right_eye = get_eyes_nose_dlib(shape)
      delta_x = right_eye[0] - left_eye[0]
      delta_y = right_eye[1] - left_eye[1]
      # angle from radians to degrees
      angle = math.atan2(delta_y, delta_x) * (180 / math.pi)
      center_of_eyes = ((left_eye[0] + right_eye[0]) // 2, (left_eye[1] + right_eye[1]) // 2)
      rotation_matrix = cv2.getRotationMatrix2D(center_of_eyes, angle, scale=1.0)

      h, w = image.shape[:2]
      aligned_image = cv2.warpAffine(image, rotation_matrix, (w, h), flags=cv2.INTER_LINEAR)

      # catch features in the image
      rotated_points = rotate_points(shape, rotation_matrix)
      eyes_image = get_eyes_in_image(aligned_image, rotated_points)
      nose_image = get_nose_in_image(aligned_image, rotated_points)
      mouth_image = get_mouth_in_image(aligned_image, rotated_points)

      # save the catched features into image
      os.makedirs("eyes", exist_ok=True)
      os.makedirs("nose", exist_ok=True)
      os.makedirs("mouth", exist_ok=True)
      if eyes_image.size == 0:
        continue
      if nose_image.size == 0:
        continue
      if mouth_image.size == 0:
        continue
      if len(file_name.split("_")) != 4:
        print(file_name)
        continue
      else:
        cv2.imwrite(os.path.join("eyes", "eyes_" + file_name), eyes_image)
        cv2.imwrite(os.path.join("nose", "nose_" + file_name), nose_image)
        cv2.imwrite(os.path.join("mouth", "mouth_" + file_name), mouth_image)

      # boolean=True, image is aligned
      return True
  return False

def main():
  folder_path = "/content/biometric_systems_project"
  for file_name in os.listdir(folder_path):

    # zipped files in the folder biometric_systems_dataset
    if file_name.endswith(".zip"):

      # full path to the zip file
      file_path = os.path.join(folder_path, file_name)

      # create folder in the same path with the name of the zip file
      extract_folder = os.path.join(folder_path, file_name.replace(".zip", ""))
      os.makedirs(extract_folder, exist_ok=True)

      # open and extract the file
      with zipfile.ZipFile(file_path, mode="r") as zip_ref:
        zip_ref.extractall(extract_folder)

  aligned_faces, total_faces = 0, 0
  dataset_path = "/content/biometric_systems_project/dataset"
  for file_name in tqdm(os.listdir(dataset_path), desc="Processing Images"):
    if not file_name.endswith(".jpg"):
      continue
    total_faces += 1
    if file_name.startswith("."):
      file_name = file_name[2:]
    image = cv2.imread(os.path.join(dataset_path, file_name))
    boolean = align_face(image, file_name)
    if boolean:
      aligned_faces += 1
  print(f"Total faces: {total_faces}")
  print(f"Aligned faces: {aligned_faces}")
  shutil.make_archive("eyes", "zip", "eyes")
  shutil.make_archive("nose", "zip", "nose")
  shutil.make_archive("mouth", "zip", "mouth")


if __name__ == "__main__":
  main()